In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import subprocess
import time

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Store previous hand position
prev_x = None
last_switch_time = time.time()

# Function to switch apps using AppleScript
def switch_app(direction):
    if direction == "right":  # Next app (Cmd + Tab)
        script = 'tell application "System Events" to key code 48 using command down'
    elif direction == "left":  # Previous app (Cmd + Shift + Tab)
        script = 'tell application "System Events" to key code 48 using {command down, shift down}'
    
    subprocess.run(["osascript", "-e", script])

# Start Video Capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert BGR to RGB for MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get landmark for the index finger tip
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

            # Convert to pixel coordinates
            h, w, _ = frame.shape
            x = int(index_tip.x * w)

            # Detect swipe gesture
            if prev_x is not None:
                movement = x - prev_x

                if abs(movement) > 100 and (time.time() - last_switch_time) > 1:  # Avoid fast switching
                    if movement > 0:
                        switch_app("right")  # Swipe right → Next app
                        print("Switched to next app")
                    else:
                        switch_app("left")  # Swipe left → Previous app
                        print("Switched to previous app")

                    last_switch_time = time.time()

            prev_x = x  # Update previous position

    # Show Camera Feed
    cv2.imshow("Hand Gesture App Switcher", frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1751889030.484968  805591 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1751889030.554997  805728 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751889030.566306  805728 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751889060.026287  805732 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
36:66: execution error: System Events got an error: osascript is not allowed to send keystrokes. (1002)


Switched to next app
Switched to previous app
Switched to next app
Switched to previous app
Switched to previous app
Switched to next app
Switched to previous app
Switched to next app
Switched to previous app
Switched to next app
Switched to previous app
Switched to next app
Switched to next app
Switched to next app
Switched to previous app
Switched to next app
Switched to previous app
Switched to next app
Switched to previous app
Switched to next app
Switched to next app
Switched to previous app
Switched to previous app
Switched to previous app
Switched to next app


KeyboardInterrupt: 

: 